In [ ]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sam
import holodeck.gravwaves
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR, SPLC

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
plt.rcParams.update({'grid.alpha': 0.5})
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode

log = holo.log
log.setLevel(logging.INFO)

# Sample Full Light-Cone Population of Binaries

THIS IS NOT WORKING YET

In [ ]:
# NOTE:  
pop = holo.population.Pop_Illustris()
fixed = holo.evolution.Fixed_Time.from_pop(pop, 2.0 * GYR)
evo = holo.evolution.Evolution(pop, fixed)
evo.evolve()

# fobs = holo.utils.nyquist_freqs(10.0 * YR, 0.2 * YR)
fobs = np.linspace(1.0, 2.0, 3) / YR

In [ ]:
nn, ss = evo._sample_universe(fobs)
print(f"{ss.shape[1]:.4e}")

In [ ]:
hs, fo = holo.sam._strains_from_samples(samples)

nloud = 5
colors = plot._get_cmap('plasma')(np.linspace(0.05, 0.95, nloud))# print(colors)

fig, ax = plot.figax(figsize=[12, 8], xlabel='Frequency [yr$^{-1}$]', ylabel='c-Strain')
for ii in utils.tqdm(range(fobs.size-1)):
    # if ii < 10 or ii > 16:
    #     continue
    
    fextr = [fobs[ii+jj] for jj in range(2)]
    fextr = np.asarray(fextr)
    cycles = 1.0 / np.diff(np.log(fextr))[0]

    idx = (fextr[0] <= fo) & (fo < fextr[1])
    hs_bin = hs[idx]
    fo_bin = fo[idx]    

    tot = np.sqrt(np.sum(cycles * hs_bin**2))
    ax.plot(fextr*YR, tot * np.ones_like(fextr), 'k--')

    idx = np.argsort(hs_bin)[::-1]
    
    for jj, cc in enumerate(colors):
        if jj > len(idx):
            break
        hi = idx[jj]
        lo = idx[jj+1:]
        gw_hi = np.sqrt(np.sum(cycles * hs_bin[hi]**2))
        gw_lo = np.sqrt(np.sum(cycles * hs_bin[lo]**2))

        fave = np.average(fo_bin[hi], weights=hs_bin[hi])
        ax.plot(fextr*YR, gw_lo * np.ones_like(fextr), color=cc, lw=0.5)
        ax.scatter(fave*YR, gw_hi, marker='.', color=cc, alpha=0.5)

plt.show()